In [1]:
import requests
import xml.etree.ElementTree as ET

def list_dataflows_insee():
    """
    Récupère la liste des regroupements thématiques (dataflows)
    exposés par l'API Insee via l'URL https://bdm.insee.fr/series/sdmx/dataflow
    Renvoie une liste de dictionnaires : [{'id':..., 'name_fr':..., 'url':...}, ...]
    """
    # 1) Faire la requête
    url = "https://bdm.insee.fr/series/sdmx/dataflow"
    response = requests.get(url)
    response.raise_for_status()

    # 2) Définir les namespaces repérés dans ta réponse XML
    ns = {
        'mes': "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
        'str': "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
        'com': "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common"
    }

    # 3) Parser la réponse
    root = ET.fromstring(response.content)

    # On va stocker les dataflows dans une liste de dict
    dataflows = []

    # 4) Trouver tous les <str:Dataflow>
    for df in root.findall(".//str:Dataflow", ns):
        # Récupération de l'ID du dataflow
        dataflow_id = df.get("id")

        # On va chercher un <com:Name xml:lang="fr"> pour le nom français
        name_fr = None
        for name_node in df.findall("com:Name", ns):
            if name_node.attrib.get("{http://www.w3.org/XML/1998/namespace}lang") == 'fr':
                name_fr = name_node.text
                break

        # On peut également choper le 1er <com:AnnotationURL> si présent
        annotation_url_nodes = df.findall(".//com:AnnotationURL", ns)
        annotation_url = annotation_url_nodes[0].text if annotation_url_nodes else None

        dataflows.append({
            'id': dataflow_id,
            'name_fr': name_fr,
            'url': annotation_url
        })

    return dataflows

if __name__ == "__main__":
    dfs = list_dataflows_insee()

    for df in dfs:
        print(df)

{'id': 'BALANCE-PAIEMENTS', 'name_fr': 'Balance des paiements', 'url': 'https://www.insee.fr/fr/statistiques/series/103212755'}
{'id': 'CHOMAGE-TRIM-NATIONAL', 'name_fr': 'Chômage, taux de chômage par sexe et âge (sens BIT)', 'url': 'https://www.insee.fr/fr/statistiques/series/103167923'}
{'id': 'CLIMAT-AFFAIRES', 'name_fr': 'Indicateurs synthétiques du climat des affaires', 'url': 'https://www.insee.fr/fr/statistiques/series/103047029'}
{'id': 'CNA-2010-CONSO-MEN', 'name_fr': 'Consommation des ménages - Résultats par produit, fonction et durabilité', 'url': 'https://www.insee.fr/fr/statistiques/series/102331845'}
{'id': 'CNA-2010-CONSO-SI', 'name_fr': 'Dépenses de consommation finale par secteur institutionnel - Résultats par opération et produit', 'url': 'https://www.insee.fr/fr/statistiques/series/102809534'}
{'id': 'CNA-2010-CPEB', 'name_fr': 'Comptes de production et d’exploitation par branche', 'url': 'https://www.insee.fr/fr/statistiques/series/102852781'}
{'id': 'CNA-2010-CSI',

In [69]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_insee_structurespecific(xml_content, remove_stopped=True):
    """
    Parse the StructureSpecific SDMX XML response from Insee
    and return a pandas DataFrame with time series observations,
    taking into account that <Series> and <Obs> are in no namespace.
    """

    root = ET.fromstring(xml_content)
    all_rows = []

    # 1) Find all <Series> elements (no namespace)
    #    e.g.  <Series IDBANK="xxx"> <Obs TIME_PERIOD="yyy" /> ...
    series_list = root.findall(".//Series")  # no namespace prefix

    for series in series_list:
        # Each <Series> can have attributes like TITLE_FR, TITLE_EN, etc.
        s_attrib = series.attrib  # dictionary of attributes on <Series>
        if s_attrib.get("SERIE_ARRETEE") == "TRUE" and remove_stopped:
            continue

        # 2) Now find all <Obs> children (also in no namespace)
        #    We'll do series.findall("./Obs") to get direct children
        obs_list = series.findall("./Obs")

        for obs in obs_list:
            # The main attributes are TIME_PERIOD, OBS_VALUE, OBS_STATUS, etc.
            time_period = obs.get("TIME_PERIOD")
            obs_value_str = obs.get("OBS_VALUE")
            obs_status = obs.get("OBS_STATUS")

            # Convert obs_value to float (unless it's 'NaN')
            if obs_value_str == "NaN":
                obs_value = None
            else:
                try:
                    obs_value = float(obs_value_str)
                except (ValueError, TypeError):
                    obs_value = None

            # Build a dict (row) for each observation
            row = {
                "TITLE_FR": s_attrib.get("TITLE_FR"),
                "TITLE_EN": s_attrib.get("TITLE_EN"),
                "LAST_UPDATE": s_attrib.get("LAST_UPDATE"),
                "IDBANK": s_attrib.get("IDBANK"),
                "INDICATEUR": s_attrib.get("INDICATEUR"),
                "CORRECTION": s_attrib.get("CORRECTION"),
                "NATURE": s_attrib.get("NATURE"),
                "UNIT": s_attrib.get("UNIT_MEASURE"),
                "MULT": s_attrib.get("UNIT_MULT"),
                "TIME_PERIOD": time_period,
                "OBS_VALUE": obs_value,
                "OBS_STATUS": obs_status,

            }
            all_rows.append(row)

    return pd.DataFrame(all_rows)


In [70]:
def fetch_insee_sdmx(url, params=None):
    if params is None:
        params = {}
    r = requests.get(url, params=params)

    if r.status_code == 200:
        return r.content
    else:
        content = r.content
        root = ET.fromstring(content)

        error_msg = root.find(".//{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}ErrorMessage")
        if error_msg is not None and error_msg.get("code") == "510":
            return content
        else:
            r.raise_for_status()

In [71]:
def parse_splitted_queries(xml_content):
    root = ET.fromstring(xml_content)

    error_msg = root.find(".//{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message}ErrorMessage")
    splitted_queries = []

    if error_msg is not None and error_msg.get("code") == "510":
        ns_com = "{http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common}"
        text_elements = error_msg.findall(f".//{ns_com}Text")

        for el in text_elements:
            text_val = el.text or ""
            text_val = text_val.strip()
            if text_val.startswith("/data/"):
                splitted_queries.append(text_val)
    return splitted_queries

In [72]:
def fetch_insee_with_fallback(base_url, params=None):
    content = fetch_insee_sdmx(base_url, params=params)

    splitted_queries = parse_splitted_queries(content)

    if splitted_queries:
        dfs = []
        for path in splitted_queries:
            # Reconstruire l'URL complète
            sub_url = "https://bdm.insee.fr/series/sdmx" + path
            sub_content = fetch_insee_sdmx(sub_url, params=params)
            df_sub = parse_insee_structurespecific(sub_content, False)
            dfs.append(df_sub)
        return pd.concat(dfs, ignore_index=True)

    else:
        return parse_insee_structurespecific(content, False)

In [73]:
url = "https://bdm.insee.fr/series/sdmx/data/BALANCE-PAIEMENTS/all"

df = fetch_insee_with_fallback(url)

print("Nombre de lignes:", len(df))

Nombre de lignes: 38809


In [60]:
df.head(500)

,TITLE_FR,TITLE_EN,LAST_UPDATE,IDBANK,INDICATEUR,CORRECTION,NATURE,UNIT,MULT,TIME_PERIOD,OBS_VALUE,OBS_STATUS
0,Taux de mortalité - Tumeurs - Hommes - Grand Est,Mortality rate - Tumours - Men - Grand Est,2024-12-09,001733750,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2022,307.2,A
1,Taux de mortalité - Tumeurs - Hommes - Grand Est,Mortality rate - Tumours - Men - Grand Est,2024-12-09,001733750,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2021,NaN,O
2,Taux de mortalité - Tumeurs - Hommes - Grand Est,Mortality rate - Tumours - Men - Grand Est,2024-12-09,001733750,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2020,304.4,A
3,Taux de mortalité - Tumeurs - Hommes - Grand Est,Mortality rate - Tumours - Men - Grand Est,2024-12-09,001733750,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2019,NaN,O
4,Taux de mortalité - Tumeurs - Hommes - Grand Est,Mortality rate - Tumours - Men - Grand Est,2024-12-09,001733750,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2018,NaN,O
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Taux de mortalité - Tumeurs - Hommes - Puy-de-...,Mortality rate - Tumours - Men - Puy-de-Dôme,2024-12-09,001733909,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2022,324.9,A
496,Taux de mortalité - Tumeurs - Hommes - Puy-de-...,Mortality rate - Tumours - Men - Puy-de-Dôme,2024-12-09,001733909,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2021,NaN,O
497,Taux de mortalité - Tumeurs - Hommes - Puy-de-...,Mortality rate - Tumours - Men - Puy-de-Dôme,2024-12-09,001733909,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2020,329.2,A
498,Taux de mortalité - Tumeurs - Hommes - Puy-de-...,Mortality rate - Tumours - Men - Puy-de-Dôme,2024-12-09,001733909,TAUX_MORTALITE,BRUT,TAUX,P100000,0,2019,NaN,O


In [63]:
import pandas as pd
import numpy as np
import re

# ------------------------------------------------------------------------------
# 1) Example of a custom function to parse the TIME_PERIOD.
#    We assume it could be:
#       - "YYYY"
#       - "YYYY-Qi"  (i from 1 to 4)
#       - "YYYY-MM"
# ------------------------------------------------------------------------------

def parse_time_period(x):
    x = str(x).strip()

    # Match YYYY only
    match_year = re.match(r'^(\d{4})$', x)
    # Match YYYY-Q1, YYYY-Q2, etc.
    match_quarter = re.match(r'^(\d{4})-Q([1-4])$', x)
    # Match YYYY-MM
    match_month = re.match(r'^(\d{4})-(\d{2})$', x)

    if match_year:
        year = int(match_year.group(1))
        # Return something like 1st Jan of that year.
        # Alternatively, you can store it as a Period with annual frequency
        # return pd.Period(str(year), freq='A')
        return pd.to_datetime(f'{year}-01-01')

    elif match_quarter:
        year = int(match_quarter.group(1))
        quarter = int(match_quarter.group(2))
        # As a date, let's assume first day of that quarter:
        month_start = 3 * (quarter - 1) + 1  # Q1=1 -> Month=1, Q2=2 -> Month=4, etc.
        return pd.to_datetime(f'{year}-{month_start:02d}-01')
        # Alternatively, you can store it as a Period:
        # return pd.Period(f'{year}Q{quarter}', freq='Q')

    elif match_month:
        year = int(match_month.group(1))
        month = int(match_month.group(2))
        # 1st day of that month
        return pd.to_datetime(f'{year}-{month:02d}-01')

    else:
        # If it doesn't match any known pattern, return NaT or None
        return pd.NaT

# ------------------------------------------------------------------------------
# 2) Let's assume df is your initial dataframe with columns:
#    ['TITLE_FR', 'IDBANK', 'CORRECTION', 'NATURE', 'UNIT', 'MULT', 'TIME_PERIOD', 'OBS_VALUE', etc...]
# ------------------------------------------------------------------------------

def transform_insee_df(df):

    # Make a copy to avoid modifying the original
    df_copy = df.copy()

    # 2a) Parse the TIME_PERIOD into a new column with a standard format
    df_copy['DATE_PARSED'] = df_copy['TIME_PERIOD'].apply(parse_time_period)

    # 2b) Convert MULT to numeric (in case it is string) and handle missing
    #     If MULT is missing or empty, let's assume 0 -> 10**0 = 1
    df_copy['MULT'] = pd.to_numeric(df_copy['MULT'], errors='coerce').fillna(0)

    # 2c) Multiply OBS_VALUE by 10**MULT
    df_copy['OBS_VALUE'] = df_copy['OBS_VALUE'] * (10 ** df_copy['MULT'])

    # ----------------------------------------------------------------------------
    # 3) Create the pivoted table:
    #    index = DATE_PARSED
    #    columns = TITLE_FR
    #    values = OBS_VALUE
    # ----------------------------------------------------------------------------

    df_pivot = df_copy.pivot_table(
        index='DATE_PARSED',
        columns='TITLE_FR',
        values='OBS_VALUE',
        aggfunc='first'  # or 'mean', or another if you expect duplicates
    )

    # ----------------------------------------------------------------------------
    # 4) Build the metadata table:
    #    We want each TITLE_FR to have a single row with [IDBANK, CORRECTION, NATURE, UNIT, ...]
    #    That means we can group or drop duplicates.
    # ----------------------------------------------------------------------------

    metadata_cols = ['TITLE_FR', 'IDBANK', 'CORRECTION', 'NATURE', 'UNIT']
    # Adjust the columns you want to keep for metadata
    df_metadata = df_copy[metadata_cols].drop_duplicates(subset='TITLE_FR').set_index('TITLE_FR')

    # Return both dataframes
    return df_pivot, df_metadata

In [74]:
from inputs import SOURCE_TABLES
from tqdm.notebook import tqdm

l = []
for name in tqdm(SOURCE_TABLES):
    url = f"https://bdm.insee.fr/series/sdmx/data/{name}/all"
    df = fetch_insee_with_fallback(url)
    df_pivoted, df_meta = transform_insee_df(df)
    l.append((df_pivoted, df_meta))

  0%|          | 0/100 [00:00<?, ?it/s]

In [76]:
all_names = []
for name, dfs in zip(SOURCE_TABLES, l):
    name = name.replace(" ","")
    df, meta = dfs
    df.to_csv(f"Data/{name}.csv")
    meta.to_csv(f"Data/{name}_meta.csv")
    all_names.append(name)

import json

with open("Data/all_data.json", 'w') as f:
    json.dump(all_names, f)

In [152]:
df_meta.head(100)

,IDBANK,CORRECTION,NATURE,UNIT
TITLE_FR,,,,
"Enquête trimestrielle de conjoncture dans l'industrie - Industrie manufacturière, industries extractives et autres - Barrières à l’embauche - Autres - Série brute",010599351,BRUT,PROPORTION,POURCENT
Enquête trimestrielle de conjoncture dans l'industrie - Industrie automobile - Barrières à l’embauche - Autres - Série brute,010599352,BRUT,PROPORTION,POURCENT
Enquête trimestrielle de conjoncture dans l'industrie - Fabrication d'autres matériels de transport - Barrières à l’embauche - Autres - Série brute,010599353,BRUT,PROPORTION,POURCENT
"Enquête trimestrielle de conjoncture dans l'industrie - Fabrication de denrées alimentaires, de boissons et de produits à base de tabac - Barrières à l’embauche - Autres - Série brute",010599354,BRUT,PROPORTION,POURCENT
"Enquête trimestrielle de conjoncture dans l'industrie - Fabrication d'équipements électriques, électroniques, informatiques ; fabrication de machines - Barrières à l’embauche - Autres - Série brute",010599355,BRUT,PROPORTION,POURCENT
...,...,...,...,...
"Enquête trimestrielle de conjoncture dans l'industrie - Industrie manufacturière, industries extractives et autres - Tendance passée de la demande globale (solde d'opinion) - Série CVS",001586501,CVS,SOLDE_PROPORTION,POURCENT
Enquête trimestrielle de conjoncture dans l'industrie - Industrie manufacturière - Tendance prévue de la demande globale (solde d'opinion) - Série brute,001586516,BRUT,SOLDE_PROPORTION,POURCENT
Enquête trimestrielle de conjoncture dans l'industrie - Industrie manufacturière - Tendance prévue de la demande globale (solde d'opinion) - Série CVS,001586517,CVS,SOLDE_PROPORTION,POURCENT


In [2]:
import json
from inputs import SOURCE_TABLES
import pandas as pd

# 1) Load all the files
with open("Data/all_data.json", 'r') as f:
    file_names = json.load(f)

file_names = [f"Data/{x}.csv" for x in file_names]

# 2) Load them into individual DataFrames
df_list = []
for file in file_names:
    temp_df = pd.read_csv(
        file,
        parse_dates=["DATE_PARSED"]  # adjust if your date column differs
    )
    df_list.append(temp_df)

# 3) Merge all DataFrames on DATE_PARSED
merged_df = df_list[0]
for i in range(1, len(df_list)):
    merged_df = pd.merge(
        merged_df,
        df_list[i],
        on="DATE_PARSED",
        how="outer"
    )

# 4) Set DATE_PARSED as index
merged_df.set_index("DATE_PARSED", inplace=True)
merged_df.sort_index(inplace=True)

# 5) Calculate the percentage of missing columns for each date
merged_df["pct_missing"] = merged_df.isna().mean(axis=1) * 100

# 6) (Optional) If you'd like to see average missingness by year/quarter/month, you can do:
# e.g., monthly
monthly_missing = merged_df.resample('M')["pct_missing"].mean()

# or yearly
yearly_missing = merged_df.resample('Y')["pct_missing"].mean()

print("Monthly % of Missing Columns:")
print(monthly_missing)

print("\nYearly % of Missing Columns:")
print(yearly_missing)

C:\Users\Paul\AppData\Local\Temp\ipykernel_26660\758196406.py:39: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_missing = merged_df.resample('M')["pct_missing"].mean()
C:\Users\Paul\AppData\Local\Temp\ipykernel_26660\758196406.py:42: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_missing = merged_df.resample('Y')["pct_missing"].mean()


Monthly % of Missing Columns:
DATE_PARSED
1901-01-31    99.984374
1901-02-28          NaN
1901-03-31          NaN
1901-04-30          NaN
1901-05-31          NaN
                ...    
2024-08-31    81.885802
2024-09-30    81.943620
2024-10-31    80.237210
2024-11-30    90.783511
2024-12-31    97.771666
Freq: ME, Name: pct_missing, Length: 1488, dtype: float64

Yearly % of Missing Columns:
DATE_PARSED
1901-12-31    99.984374
1902-12-31    99.984374
1903-12-31    99.984374
1904-12-31    99.984374
1905-12-31    99.984374
                ...    
2020-12-31    70.949490
2021-12-31    70.611047
2022-12-31    73.138237
2023-12-31    74.732136
2024-12-31    80.093316
Freq: YE-DEC, Name: pct_missing, Length: 124, dtype: float64


In [10]:
monthly_missing.iloc[750:].describe()

count    738.000000
mean      86.454792
std       15.504850
min        9.030534
25%       79.071475
50%       90.355346
75%       97.873238
max       99.990624
Name: pct_missing, dtype: float64